Adapted from [Software Carpentry](https://swcarpentry.github.io/python-novice-inflammation/08-func/index.html "08-func")

---




# Creating Functions


teaching: 30


exercises: 0


questions:


- "How can I define new functions?"
- "What's the difference between defining and calling a function?"
- "What happens when I call a function?"


objectives:


- "Define a function that takes parameters."
- "Return a value from a function."
- "Test and debug a function."
- "Set default values for function parameters."
- "Explain why we should divide programs into small, single-purpose functions."


keypoints:


- "Define a function using `def function_name(parameter)`."
- "The body of a function must be indented."
- "Call a function using `function_name(value)`."
- "Numbers are stored as integers or floating-point numbers."
- "Variables defined within a function can only be seen and used within the body of the function."
- "If a variable is not defined within the function it is used, Python looks for a definition before the function call"
- "Use `help(thing)` to view help for something."
- "Put docstrings in functions to provide help for that function."
- "Specify default values for parameters when defining a function using `name=value` in the parameter list."
- "Parameters can be passed by matching based on name, by position, or by omitting them (in which case the default value is used)."
- "Put code whose parameters change frequently in a function, then call it with different parameter values to customize its behavior."
---




At this point, we've written code to draw some interesting features in our inflammation data, loop over all our data files to quickly draw these plots for each of them, and have Python make decisions based on what it sees in our data. But, our code is getting pretty long and complicated; what if we had thousands of datasets, and didn't want to generate a figure for every single one? Commenting out the figure-drawing code is a nuisance. Also, what if we want to use that code again, on a different dataset or at a different point in our program? Cutting and pasting it is going to make our code get very long and very repetitive, very quickly. We'd like a way to package our code so that it is easier to reuse, and Python provides for this by letting us define things called 'functions' --- a shorthand way of re-executing longer pieces of code. Let's start by defining a function `fahr_to_celsius` that converts temperatures from Fahrenheit to Celsius:




In [ ]:
def fahr_to_celsius(temp):
    return ((temp - 32) * (5/9))

> <img src=fig/python-function.svg>Labeled parts of a Python function definition


The function definition opens with the keyword `def` followed by the name of the function (`fahr_to_celsius`) and a parenthesized list of parameter names (`temp`). The <span style="color:red" title="(of a function): the statements that are executed when a function runs.">body</span> of the function --- the statements that are executed when it runs --- is indented below the definition line.  The body concludes with a `return` keyword followed by the return value.


When we call the function, the values we pass to it are assigned to those variables so that we can use them inside the function. Inside the function, we use a <span style="color:red" title="A statement that causes a function to stop executing and return a value to its caller immediately.">return statement</span> to send a result back to whoever asked for it.


Let's try running our function.




In [ ]:
fahr_to_celsius(32)

This command should call our function, using "32" as the input and return the function value.


In fact, calling our own function is no different from calling any other function:




In [ ]:
print('freezing point of water:', fahr_to_celsius(32), 'C')
print('boiling point of water:', fahr_to_celsius(212), 'C')

We've successfully called the function that we defined, and we have access to the value that we returned.


## Composing Functions


Now that we've seen how to turn Fahrenheit into Celsius, we can also write the function to turn Celsius into Kelvin:




In [ ]:
def celsius_to_kelvin(temp_c):
    return temp_c + 273.15

print('freezing point of water in Kelvin:', celsius_to_kelvin(0.))

What about converting Fahrenheit to Kelvin? We could write out the formula, but we don't need to. Instead, we can <span style="color:red" title="To apply one function to the result of another, such as `f(g(x))`.">compose</span> the two functions we have already created:




In [ ]:
def fahr_to_kelvin(temp_f):
    temp_c = fahr_to_celsius(temp_f)
    temp_k = celsius_to_kelvin(temp_c)
    return temp_k

print('boiling point of water in Kelvin:', fahr_to_kelvin(212.0))

This is our first taste of how larger programs are built: we define basic operations, then combine them in ever-larger chunks to get the effect we want. Real-life functions will usually be larger than the ones shown here --- typically half a dozen to a few dozen lines --- but they shouldn't ever be much longer than that, or the next person who reads it won't be able to understand what's going on.


## Tidying up


Now that we know how to wrap bits of code up in functions, we can make our inflammation analysis easier to read and easier to reuse. First, let's make a `visualize` function that generates our plots:




In [ ]:
import numpy
import matplotlib.pyplot

def visualize(filename):

    data = numpy.loadtxt(fname=filename, delimiter=',')

    fig = matplotlib.pyplot.figure(figsize=(10.0, 3.0))

    axes1 = fig.add_subplot(1, 3, 1)
    axes2 = fig.add_subplot(1, 3, 2)
    axes3 = fig.add_subplot(1, 3, 3)

    axes1.set_ylabel('average')
    axes1.plot(numpy.mean(data, axis=0))

    axes2.set_ylabel('max')
    axes2.plot(numpy.max(data, axis=0))

    axes3.set_ylabel('min')
    axes3.plot(numpy.min(data, axis=0))

    fig.tight_layout()
    matplotlib.pyplot.show()

and another function called `detect_problems` that checks for those systematics we noticed:




In [ ]:
def detect_problems(filename):

    data = numpy.loadtxt(fname=filename, delimiter=',')

    if numpy.max(data, axis=0)[0] == 0 and numpy.max(data, axis=0)[20] == 20:
        print('Suspicious looking maxima!')
    elif numpy.sum(numpy.min(data, axis=0)) == 0:
        print('Minima add up to zero!')
    else:
        print('Seems OK!')

Wait! Didn't we forget to specify what both of these functions should return? Well, we didn't. In Python, functions are not required to include a `return` statement and can be used for the sole purpose of grouping together pieces of code that conceptually do one thing. In such cases, function names usually describe what they do, _e.g._ `visualize`, `detect_problems`.


Notice that rather than jumbling this code together in one giant `for` loop, we can now read and reuse both ideas separately. We can reproduce the previous analysis with a much simpler `for` loop:




In [ ]:
import glob

filenames = sorted(glob.glob('data/inflammation*.csv'))

for filename in filenames[:3]:
    print(filename)
    visualize(filename)
    detect_problems(filename)

By giving our functions human-readable names, we can more easily read and understand what is happening in the `for` loop. Even better, if at some later date we want to use either of those pieces of code again, we can do so in a single line.


## Testing and Documenting


Once we start putting things in functions so that we can re-use them, we need to start testing that those functions are working correctly. To see how to do this, let's write a function to offset a dataset so that it's mean value shifts to a user-defined value:




In [ ]:
def offset_mean(data, target_mean_value):
    return (data - numpy.mean(data)) + target_mean_value

We could test this on our actual data, but since we don't know what the values ought to be, it will be hard to tell if the result was correct. Instead, let's use NumPy to create a matrix of 0's and then offset its values to have a mean value of 3:




In [ ]:
z = numpy.zeros((2,2))
print(offset_mean(z, 3))

That looks right, so let's try `offset_mean` on our real data:




In [ ]:
data = numpy.loadtxt(fname='data/inflammation-01.csv', delimiter=',')
print(offset_mean(data, 0))

It's hard to tell from the default output whether the result is correct, but there are a few tests that we can run to reassure us:




In [ ]:
print('original min, mean, and max are:', numpy.min(data), numpy.mean(data), numpy.max(data))
offset_data = offset_mean(data, 0)
print('min, mean, and max of offset data are:',
      numpy.min(offset_data),
      numpy.mean(offset_data),
      numpy.max(offset_data))

That seems almost right: the original mean was about 6.1, so the lower bound from zero is now about -6.1. The mean of the offset data isn't quite zero --- we'll explore why not in the challenges --- but it's pretty close. We can even go further and check that the standard deviation hasn't changed:




In [ ]:
print('std dev before and after:', numpy.std(data), numpy.std(offset_data))

Those values look the same, but we probably wouldn't notice if they were different in the sixth decimal place. Let's do this instead:




In [ ]:
print('difference in standard deviations before and after:',
      numpy.std(data) - numpy.std(offset_data))

Again, the difference is very small. It's still possible that our function is wrong, but it seems unlikely enough that we should probably get back to doing our analysis. We have one more task first, though: we should write some <span style="color:red" title="Human-language text written to explain what software does, how it works, or how to use it.">documentation</span> for our function to remind ourselves later what it's for and how to use it.


The usual way to put documentation in software is to add <span style="color:red" title="A remark in a program that is intended to help human readers understand what is going on, but is ignored by the computer. Comments in Python, R, and the Unix shell start with a `#` character and run to the end of the line; comments in SQL start with `--`, and other languages have other conventions.">comments</span> like this:




In [ ]:
# offset_mean(data, target_mean_value):
# return a new array containing the original data with its mean offset to match the desired value.
def offset_mean(data, target_mean_value):
    return (data - numpy.mean(data)) + target_mean_value

There's a better way, though. If the first thing in a function is a string that isn't assigned to a variable, that string is attached to the function as its documentation:




In [ ]:
def offset_mean(data, target_mean_value):
    """Return a new array containing the original data
       with its mean offset to match the desired value."""
    return (data - numpy.mean(data)) + target_mean_value

This is better because we can now ask Python's built-in help system to show us the documentation for the function:




In [ ]:
help(offset_mean)

A string like this is called a <span style="color:red" title="Short for 'documentation string', this refers to textual documentation embedded in Python programs. Unlike comments, docstrings are preserved in the running program and can be examined in interactive sessions.">docstring</span>. We don't need to use triple quotes when we write one, but if we do, we can break the string across multiple lines:




In [ ]:
def offset_mean(data, target_mean_value):
    """Return a new array containing the original data
       with its mean offset to match the desired value.

    Examples
    --------
    >>> offset_mean([1, 2, 3], 0)
    array([-1.,  0.,  1.])
    """
    return (data - numpy.mean(data)) + target_mean_value

help(offset_mean)

## Defining Defaults


We have passed parameters to functions in two ways: directly, as in `type(data)`, and by name, as in `numpy.loadtxt(fname='something.csv', delimiter=',')`. In fact, we can pass the filename to `loadtxt` without the `fname=`:




In [ ]:
numpy.loadtxt('data/inflammation-01.csv', delimiter=',')

but we still need to say `delimiter=`:




In [ ]:
numpy.loadtxt('data/inflammation-01.csv', ',')

To understand what's going on, and make our own functions easier to use, let's re-define our `offset_mean` function like this:




In [ ]:
def offset_mean(data, target_mean_value=0.0):
    """Return a new array containing the original data
       with its mean offset to match the desired value, (0 by default).

    Examples
    --------
    >>> offset_mean([1, 2, 3])
    array([-1.,  0.,  1.])
    """
    return (data - numpy.mean(data)) + target_mean_value

The key change is that the second parameter is now written `target_mean_value=0.0` instead of just `target_mean_value`. If we call the function with two arguments, it works as it did before:




In [ ]:
test_data = numpy.zeros((2, 2))
print(offset_mean(test_data, 3))

But we can also now call it with just one parameter, in which case `target_mean_value` is automatically assigned the <span style="color:red" title="A value to use for a parameter if nothing is specified explicitly.">default value</span> of 0.0:




In [ ]:
more_data = 5 + numpy.zeros((2, 2))
print('data before mean offset:')
print(more_data)
print('offset data:')
print(offset_mean(more_data))

This is handy: if we usually want a function to work one way, but occasionally need it to do something else, we can allow people to pass a parameter when they need to but provide a default to make the normal case easier. The example below shows how Python matches values to parameters:




In [ ]:
def display(a=1, b=2, c=3):
    print('a:', a, 'b:', b, 'c:', c)

print('no parameters:')
display()
print('one parameter:')
display(55)
print('two parameters:')
display(55, 66)

As this example shows, parameters are matched up from left to right, and any that haven't been given a value explicitly get their default value. We can override this behavior by naming the value as we pass it in:




In [ ]:
print('only setting the value of c')
display(c=77)

With that in hand, let's look at the help for `numpy.loadtxt`:




In [ ]:
help(numpy.loadtxt)

There's a lot of information here, but the most important part is the first couple of lines:


This tells us that `loadtxt` has one parameter called `fname` that doesn't have a default value, and eight others that do. If we call the function like this:




In [ ]:
numpy.loadtxt('data/inflammation-01.csv', ',')

then the filename is assigned to `fname` (which is what we want), but the delimiter string `','` is assigned to `dtype` rather than `delimiter`, because `dtype` is the second parameter in the list. However `','` isn't a known `dtype` so our code produced an error message when we tried to run it. When we call `loadtxt` we don't have to provide `fname=` for the filename because it's the first item in the list, but if we want the `','` to be assigned to the variable `delimiter`, we *do* have to provide `delimiter=` for the second parameter since `delimiter` is not the second parameter in the list.


## Readable functions


Consider these two functions:




In [ ]:
def s(p):
    a = 0
    for v in p:
        a += v
    m = a / len(p)
    d = 0
    for v in p:
        d += (v - m) * (v - m)
    return numpy.sqrt(d / (len(p) - 1))

def std_dev(sample):
    sample_sum = 0
    for value in sample:
        sample_sum += value

    sample_mean = sample_sum / len(sample)

    sum_squared_devs = 0
    for value in sample:
        sum_squared_devs += (value - sample_mean) * (value - sample_mean)

    return numpy.sqrt(sum_squared_devs / (len(sample) - 1))

The functions `s` and `std_dev` are computationally equivalent (they both calculate the sample standard deviation), but to a human reader, they look very different. You probably found `std_dev` much easier to read and understand than `s`.


As this example illustrates, both documentation and a programmer's _coding style_ combine to determine how easy it is for others to read and understand the programmer's code. Choosing meaningful variable names and using blank spaces to break the code into logical "chunks" are helpful techniques for producing _readable code_. This is useful not only for sharing code with others, but also for the original programmer. If you need to revisit code that you wrote months ago and haven't thought about since then, you will appreciate the value of readable code!


## Combining Strings


"Adding" two strings produces their concatenation: `'a' + 'b'` is `'ab'`. Write a function called `fence` that takes two parameters called `original` and `wrapper` and returns a new string that has the wrapper character at the beginning and end of the original. A call to your function should look like this:


---


<span style="color:blue">In [1]:</span>
```python
print(fence('name', '*'))
```
---




---


<span style="color:red">Out [1]:</span>
```python
*name*
```
---






In [ ]:
#Your answer

<details>  
<summary><b> Solution</b></summary>  

---  
<span style="color:blue">In [1]:</span>  
```python
def fence(original, wrapper):  
    return wrapper + original + wrapper  
```
---  


</details>  



## Return versus print


Note that `return` and `print` are not interchangeable. `print` is a Python function that *prints* data to the screen. It enables us, *users*, see the data. `return` statement, on the other hand, makes data visible to the program. Let's have a look at the following function:




In [ ]:
def add(a, b):
    print(a + b)

**Question**: What will we see if we execute the following commands?


---


<span style="color:blue">In [1]:</span>
```python
A = add(7, 3)
print(A)
```
---






<details>  
<summary><b> Solution</b></summary>  

Python will first execute the function `add` with `a = 7` and `b = 3`, and, therefore, print `10`. However, because function `add` does not have a line that starts with `return` (no `return` "statement"), it will, by default, return nothing which, in Python world, is called `None`. Therefore, `A` will be assigned to `None` and the last line (`print(A)`) will print `None`. As a result, we will see:  


---  
<span style="color:red">Out [1]:</span>  
```
10  
None  
```
---  


</details>  



## Selecting Characters From Strings


If the variable `s` refers to a string, then `s[0]` is the string's first character and `s[-1]` is its last. Write a function called `outer` that returns a string made up of just the first and last characters of its input. A call to your function should look like this:


---


<span style="color:blue">In [1]:</span>
```python
print(outer('helium'))
```
---




---


<span style="color:red">Out [1]:</span>
```python
hm
```
---






In [ ]:
#Your answer

<details>  
<summary><b> Solution</b></summary>  

---  
<span style="color:blue">In [1]:</span>  
```python
def outer(input_string):  
    return input_string[0] + input_string[-1]  
```
---  


</details>  



## Rescaling an Array


Write a function `rescale` that takes an array as input and returns a corresponding array of values scaled to lie in the range 0.0 to 1.0. (Hint: If `L` and `H` are the lowest and highest values in the original array, then the replacement for a value `v` should be `(v-L) / (H-L)`.)




In [ ]:
#Your answer

<details>  
<summary><b> Solution</b></summary>  

---  
<span style="color:blue">In [1]:</span>  
```python
def rescale(input_array):  
    L = numpy.min(input_array)  
    H = numpy.max(input_array)  
    output_array = (input_array - L) / (H - L)  
    return output_array  
```
---  


</details>  



## Testing and Documenting Your Function


Run the commands `help(numpy.arange)` and `help(numpy.linspace)` to see how to use these functions to generate regularly-spaced values, then use those values to test your `rescale` function. Once you've successfully tested your function, add a docstring that explains what it does.




In [ ]:
#Your answer

<details>  
<summary><b> Solution</b></summary>  

---  
<span style="color:blue">In [1]:</span>  
```python
"""Takes an array as input, and returns a corresponding array scaled so  
that 0 corresponds to the minimum and 1 to the maximum value of the input array.  

Examples:  
 rescale(numpy.arange(10.0))  
array([ 0.        ,  0.11111111,  0.22222222,  0.33333333,  0.44444444,  
       0.55555556,  0.66666667,  0.77777778,  0.88888889,  1.        ])  
 rescale(numpy.linspace(0, 100, 5))  
array([ 0.  ,  0.25,  0.5 ,  0.75,  1.  ])  
"""  
```
---  


</details>  



## Defining Defaults


Rewrite the `rescale` function so that it scales data to lie between `0.0` and `1.0` by default, but will allow the caller to specify lower and upper bounds if they want. Compare your implementation to your neighbor's: do the two functions always behave the same way?




In [ ]:
#Your answer

<details>  
<summary><b> Solution</b></summary>  

---  
<span style="color:blue">In [1]:</span>  
```python
def rescale(input_array, low_val=0.0, high_val=1.0):  
    """rescales input array values to lie between low_val and high_val"""  
    L = numpy.min(input_array)  
    H = numpy.max(input_array)  
    intermed_array = (input_array - L) / (H - L)  
    output_array = intermed_array * (high_val - low_val) + low_val  
    return output_array  
```
---  


</details>  



## Variables Inside and Outside Functions


What does the following piece of code display when run --- and why?


---


<span style="color:blue">In [1]:</span>
```python
f = 0
k = 0

def f2k(f):
    k = ((f-32)*(5.0/9.0)) + 273.15
    return k

f2k(8)
f2k(41)
f2k(32)

print(k)
```
---






<details>  
<summary><b> Solution</b></summary>  

---  
<span style="color:red">Out [1]:</span>  
```
259.81666666666666  
287.15  
273.15  
0  
```
---  


`k` is 0 because the `k` inside the function `f2k` doesn't know about the `k` defined outside the function.  


</details>  



## Mixing Default and Non-Default Parameters


Given the following code:


---


<span style="color:blue">In [1]:</span>
```python
def numbers(one, two=2, three, four=4):
    n = str(one) + str(two) + str(three) + str(four)
    return n

print(numbers(1, three=3))
```
---




what do you expect will be printed?  What is actually printed? What rule do you think Python is following?


1.  `1234`

2.  `one2three4`

3.  `1239`

4.  `SyntaxError`

Given that, what does the following piece of code display when run?




<details>  
<summary><b> Solution</b></summary>  

Attempting to define the `numbers` function results in `4. SyntaxError`. The defined parameters `two` and `four` are given default values. Because `one` and `three` are not given default values, they are required to be included as arguments when the function is called and must be placed before any parameters that have default values in the function definition.  


</details>  



In [ ]:
def func(a, b=3, c=6):
    print('a: ', a, 'b: ', b, 'c:', c)

func(-1, 2)

1. `a: b: 3 c: 6`

2. `a: -1 b: 3 c: 6`

3. `a: -1 b: 2 c: 6`

4. `a: b: -1 c: 2`



<details>  
<summary><b> Solution</b></summary>  

The given call to `func` displays `a: -1 b: 2 c: 6`. -1 is assigned to the first parameter `a`, 2 is assigned to the next parameter `b`, and `c` is not passed a value, so it uses its default value 6.  


</details>  



## The Old Switcheroo


Consider this code:


---


<span style="color:blue">In [1]:</span>
```python
a = 3
b = 7

def swap(a, b):
    temp = a
    a = b
    b = temp

swap(a, b)

print(a, b)
```
---




Which of the following would be printed if you were to run this code? Why did you pick this answer?


1. `7 3`

2. `3 7`

3. `3 3`

4. `7 7`



<details>  
<summary><b> Solution</b></summary>  

`3 7` is the correct answer. Initially, `a` has a value of 3 and `b` has a value of 7. When the `swap` function is called, it creates local variables (also called `a` and `b` in this case) and trades their values. The function does not return any values and does not alter `a` or `b` outside of its local copy. Therefore the original values of `a` and `b` remain unchanged.  


</details>  



## Readable Code


Revise a function you wrote for one of the previous exercises to try to make the code more readable.




In [ ]:
#Your answer